In [1]:
import pandas as pd
import numpy as np


import matplotlib.pyplot as plt

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split


from nets_algo import *

## Функция потерь

Создание функции потерь в torch
https://neptune.ai/blog/pytorch-loss-functions. Для задач бинарной классификации рекомендуют использовать CrossEntropy функцию потерь

In [2]:
loss_fn = nn.BCELoss()

# Загрузка и подговтовка данных

In [3]:
data = pd.read_csv(
    "/home/dranik/KFA/university/proc_2.csv", 
    index_col = 0
)

save_ind = data[data['Y'] == 0].sample(
    sum(data['Y']), random_state = 0
).index.union(data[data['Y'] == 1].index)
data = data.loc[save_ind]

Нужно провести One Hot Encoding

In [4]:
Y = np.array(data[['Y']])
X = data.drop('Y', axis = 1)

X = np.concatenate([
    OneHotEncoder(sparse = False).\
    fit_transform(X.loc[:,X.dtypes == "O"]),
    X.loc[:,X.dtypes != "O"].to_numpy()
], axis = 1)

Y.shape

(13926, 1)

Разбивка на Train/Test

In [5]:
X_train, X_test, y_train, y_test = \
    train_test_split(
        X,Y, random_state = 0, stratify = Y
)

## Создание набора данных и загрузчика данных

In [6]:
train_data = My_data_set(
    torch.tensor(X_train.astype('float32')), 
    torch.tensor(y_train.astype('float32'))
)

In [15]:
train_data_loader =\
torch.utils.data.DataLoader(
    train_data, batch_size=1000
)

# Различные варианты модели

In [30]:
[learning_info, auc_info] = \
model_fit_get_perfomance(
    [[6]], 50, loss_fn,
    train_data_loader, X_test, y_test,
)
auc_info

[6] AUC = 0.6234293969709156


[6]    0.623429
dtype: float64

## Модели с одним скрытым слоем

### Обобщенный построитель модели с одним слоем

In [16]:
def one_layer_fit(
    fit_range, epoches,
    file_name, **kwargs
):
    for i in fit_range:
        model_fit_get_perfomance(
            [[i]], epoches, loss_fn, 
            train_data_loader, X_test, y_test, 
            file_name, **kwargs
        )

Базовый вариант

In [17]:
one_layer_fit(
    range(2,20), 50,
    'fiting_results2/one_hiden_initial/',
    lr_decr = 1.1
)

[2] AUC = 0.6445509044463015
[3] AUC = 0.5047194383785851
[4] AUC = 0.6044407919154914
[5] AUC = 0.6465135705710273
[6] AUC = 0.6421611629646321
[7] AUC = 0.6182863473460457
[8] AUC = 0.6786037720536007
[9] AUC = 0.6766602410163238
[10] AUC = 0.6623316895853327
[11] AUC = 0.5058855240094211
[12] AUC = 0.6187350321551948
[13] AUC = 0.6779271157715681
[14] AUC = 0.6294835076990685
[15] AUC = 0.6364986616985822
[16] AUC = 0.6345787196053158
[17] AUC = 0.5834212282680667
[18] AUC = 0.68454719619832
[19] AUC = 0.6760997149201886


После анализа рисунков становиться понятно, что для посторения модели недостаточно эпох. Проделаем эту операцию с большим количесвом эпох.

In [14]:
one_layer_fit(
    range(2,20), 200,
    'fiting_results2/one_hiden_more_epoches/',
    lr_decr = 1.1, 
    lc_param = {}
)

[2] AUC = 0.4941732009141293
[3] AUC = 0.506539251178045
[4] AUC = 0.5574456769713512
[5] AUC = 0.5188513602902727
[6] AUC = 0.5278659659705564
[7] AUC = 0.5136827422295874
[8] AUC = 0.5108748331040972
[9] AUC = 0.5530449367733822
[10] AUC = 0.49250267478830156
[11] AUC = 0.4962109227697973
[12] AUC = 0.627756895972097
[13] AUC = 0.5106302009085207
[14] AUC = 0.506778604728808
[15] AUC = 0.610615486686103
[16] AUC = 0.5420709641213811
[17] AUC = 0.5090195544097965
[18] AUC = 0.649550935788255
[19] AUC = 0.6105755669347008


## Модели с двумя скрытыми слоями

In [82]:
model_fit_get_perfomance(
    [[2,2], [2,3], [2,4]], 50, train_data_loader, 
    X_test, y_test, 'fiting_results/two_layers_initial/',
    lr_decr = 2
)

[2, 2] AUC = 0.5772943712160777
[2, 3] AUC = 0.5
[2, 4] AUC = 0.5


hello    5
dtype: int64